# Download

## Fetch structures obtained by x-ray

In [18]:
import os
import time
import datetime
import textwrap
import urllib
import shutil
import glob
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor

chunk_runtime = {}

## Pick the sample
Download the structures listed on the file`list.txt`.

In [19]:
# number of structures to get from list.txt 
# 584 is the max
n_sample = 10

In [20]:
with open('list.txt') as f:
    file = f.read()
    sample = file.split('\n')

sample = sample[:n_sample]
sample_str = ', '.join(sample)

data_directory = f'{n_sample}-structures-{datetime.date.today()}'
data_structure = [
    {'dirname':'pdb',
     'filename':'{query_id}_RCSB.pdb',
     'url':'https://files.rcsb.org/download/{query_id}.pdb',
    },
    {'dirname':'pdb',
     'filename':'{query_id}_REDO.pdb',
     'url':'https://pdb-redo.eu/db/{query_id}/{query_id}_final.pdb',
    },
    {'dirname':'mtz',
    'filename':'{query_id}_REDO.mtz',
    'url':'https://pdb-redo.eu/db/{query_id}/{query_id}_final.mtz'
    }
]

print('the following structures are going to be downloaded:')
print(textwrap.fill(sample_str, 80))

the following structures are going to be downloaded:
3ato, 5v4g, 4h8y, 2ybj, 1h87, 4etd, 4lt0, 4tws, 3zvq, 3wu9


## Download
Downloading `.pdb` from RCSB and REDO, and `.mtz` from REDO.

In [21]:
start_time = datetime.timedelta(seconds=time.time())

# Directory structure
data_directory_x = data_directory
dir_num = 1
while data_directory_x in os.listdir():
    data_directory_x = data_directory + f'_{dir_num}'
    dir_num += 1

os.mkdir(data_directory_x)

for each in data_structure:
    if each['dirname'] not in os.listdir(data_directory_x):
        subdir = f'{data_directory_x}/{each["dirname"]}'
        os.mkdir(subdir)

# Downloading sample
n_downloaded = n_sample
def download_whole_data_structure(ID):
    downloaded = []
    with ThreadPoolExecutor() as thread_pool:
        for each in data_structure:
            request_url = each['url'].format(query_id=ID)
            filename = each['filename'].format(query_id=ID)
            path = f'{data_directory_x}/{each["dirname"]}/{filename}'
            download_request = thread_pool.submit(urllib.request.urlretrieve, request_url, path)
            downloaded.append([filename, download_request])
    exception_map = map(lambda row:[row[0], str(row[1].exception())], downloaded)
    return [row for row in exception_map if row[1] != 'None']

process_list = []
with ProcessPoolExecutor() as process_pool:
    for ID in sample:
        request = process_pool.submit(download_whole_data_structure, ID)
        process_list.append(request)

exception_list = []
for ID in process_list:
    results = ID.result()
    for each in results:
        exception_list.append(each)
    
n_not_downloaded = len(set(row[0][:4] for row in exception_list))
n_downloaded -= n_not_downloaded
if n_not_downloaded > 0:
    print(f'{n_not_downloaded} structures were not fully downloaded due to error!')
else:
    print('Every structure were fully downloaded!')

end_time = datetime.timedelta(seconds=time.time())

# report
n = n_sample
chunk_runtime['download'] = end_time - start_time

print(f"\nduration:\t{chunk_runtime['download']}\nstructures:\t{n_downloaded}/{n}\n")
for ID in exception_list:
      print(f'[{ID[1]}] {ID[0]} was not downloaded!')

Every structure were fully downloaded!

duration:	0:00:12.859177
structures:	10/10



# Phenix
1. use the `.mtz` files to generate the REDO and RCSB phases with `phenix.reciprocal_space_arrays`.
2. save the log from the `phenix.reciprocal_space_arrays` routine
3. convert `.mtz` file to `.cif` using `phenix.mtz_as_cif`

## Routines

In [22]:
def reciprocal_space_arrays(pdb_input, mtz_input, mtz_output, log_output):
    first_instruction = f'phenix.reciprocal_space_arrays hkl_file={mtz_input} ' \
                        f'pdb_file={pdb_input} hendrickson_lattman_coefficients_label=None ' \
                        f'remove_f_obs_outliers=True bulk_solvent_and_scaling=True ' \
                        f'output_file_name={mtz_output} > {log_output}'
    os.system(first_instruction)

def mtz_as_cif(mtz_input, cif_output):
    first_instruction = f'phenix.mtz_as_cif mtz_file={mtz_input} output_file={cif_output} ' \
                        'mtz_labels="FOBS SIGFOBS FMODEL PHIFMODEL FOM RESOLUTION" ' \
                        'cif_labels="_refln.FOBS _refln.SIGFOBS _refln.FMODEL _refln.PHIMODEL _refln.FOM _refln.RESOL"'    
    os.system(first_instruction)

## Running `phenix.reciprocal_space_arrays`

In [23]:
sample = set(sample)
exception_list = set(x[0][:4] for x in exception_list)
sample = sample - exception_list

# define the phenix log output dirname
log_output_dirname = 'phenix'

mtz_dir = data_directory_x + f'/{data_structure[2]["dirname"]}/'
mtz_input_template = mtz_dir + data_structure[2]['filename']
mtz_files = [mtz_input_template.format(query_id=each) for each in sample]

mtz_output_dirname = mtz_dir
mtz_output_template = mtz_output_dirname + '{query_id}_{source}_temporary.mtz'

log_output_dirname = f'{data_directory_x}/{log_output_dirname}/'
log_output_template = log_output_dirname + '{query_id}_{source}_phenix.log'
os.mkdir(log_output_dirname)

pdb_structure = data_structure[0]
pdb_input_template = f"{data_directory_x}/{pdb_structure['dirname']}" + '/{query_id}_{source}.pdb'

start_time = datetime.timedelta(seconds=time.time())

source_structure = ['RCSB', 'REDO']
process_list = []
with ProcessPoolExecutor() as process_pool:
    for src in source_structure: 
        for ID in sample:
            pdb_input = pdb_input_template.format(query_id=ID, source=src)
            mtz_input = mtz_input_template.format(query_id=ID)
            mtz_output = mtz_output_template.format(query_id=ID, source=src)
            log_output = log_output_template.format(query_id=ID, source=src)
            
            request = process_pool.submit(reciprocal_space_arrays, pdb_input, mtz_input, mtz_output, log_output)
            process_list.append(request)
    
end_time = datetime.timedelta(seconds=time.time())

#report
chunk_runtime['reciprocal_space_arrays'] = end_time - start_time

print(f"duration:\t{chunk_runtime['reciprocal_space_arrays']}")

duration:	0:01:17.770954


Next task: convert the `.mtz` files to `.cif`  
Defining the directory structure for this task

## Running `phenix.mtz_as_cif`

In [24]:
# define the output dirname
cif_output_dirname = 'phases'
cif_output_dirname = f'{data_directory_x}/{cif_output_dirname}/'
cif_output_template = cif_output_dirname + '{query_id}_{source}_phases.cif'
os.mkdir(cif_output_dirname)
mtztemporary_input_template = mtz_output_template


start_time = datetime.timedelta(seconds=time.time())

source_structure = ['RCSB', 'REDO']

with ProcessPoolExecutor() as process_pool:
    for src in source_structure: 
        for ID in sample:
            mtz_input = mtztemporary_input_template.format(query_id=ID, source=src)
            cif_output = cif_output_template.format(query_id=ID, source=src)
            request = process_pool.submit(mtz_as_cif, mtz_input, cif_output)
            
end_time = datetime.timedelta(seconds=time.time())

#report
chunk_runtime['mtz_as_cif'] = end_time - start_time
print(f"duration:\t{chunk_runtime['mtz_as_cif']}")

duration:	0:00:33.854936


finally, move all relevant data to the `data/` folder

In [25]:
os.mkdir("data")

for file in glob.glob(data_directory_x + r'/pdb/*_RCSB.pdb'):
    shutil.copy(file, "data/")

for file in glob.glob(data_directory_x + r'/phases/*_REDO_phases.cif'):
    shutil.copy(file, "data/")

shutil.rmtree(data_directory, ignore_errors=True)